In [1]:
import csv
import time

In [2]:
print()

In [3]:
import numpy as np

In [4]:
import torch

In [5]:
import pytorch_lightning as pl

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
from chemprop.v2 import data

In [8]:
from chemprop.v2 import featurizers

In [9]:
from chemprop.v2.models import modules, models

In [10]:
from pytorch_lightning.callbacks.progress import ProgressBar, RichProgressBar, TQDMProgressBar
import sys
class MyProgressBar(TQDMProgressBar):
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_predict_tqdm(self):
        bar = super().init_predict_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

    def init_test_tqdm(self):
        bar = super().init_test_tqdm()
        if not sys.stdout.isatty():
            bar.disable = True
        return bar

In [11]:
with open("/home/gridsan/nmorgan/MW.csv") as fid:
    reader = csv.reader(fid)
    next(reader)
    smis, scores = zip(*[(smi, float(score)) for smi, score in reader])
scores = np.array(scores).reshape(-1,1)
all_data = [data.MoleculeDatapoint(smi, target) for smi, target in zip(smis, scores)]

In [12]:
generator = torch.Generator().manual_seed(12)
[0.81,0.09,0.1]
train_data, val_data, test_data = torch.utils.data.random_split(all_data, [0.5,0.5,0], generator=generator)

featurizer = featurizers.MoleculeFeaturizer()

train_dset = data.MoleculeDataset(train_data, featurizer)
val_dset = data.MoleculeDataset(val_data, featurizer)
test_dset = data.MoleculeDataset(test_data, featurizer)

train_loader = data.MolGraphDataLoader(train_dset, num_workers=4)
val_loader = data.MolGraphDataLoader(val_dset, num_workers=4, shuffle=False)
test_loader = data.MolGraphDataLoader(test_dset, num_workers=4, shuffle=False)

/home/gridsan/nmorgan/mambaforge/envs/chemprop_gpu/lib/python3.11/site-packages/torch/utils/data/dataset.py:348: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


In [13]:
scaler = StandardScaler().fit(np.array([d._targets for d in train_dset.data]))
molenc = modules.molecule_block()
mpnn = models.RegressionMPNN(mpn_block = molenc, 
                             n_tasks = 1, 
                             init_lr = 5e-3,
                             max_lr = 1e-2,
                             final_lr = 5e-3,
                             scaler = scaler,
                             )

In [14]:
if torch.cuda.is_available():
    trainer = pl.Trainer(
        enable_progress_bar=True,
        devices=1,
        max_epochs=200,
        callbacks=[MyProgressBar()],
        accelerator="gpu",
        default_root_dir="/home/gridsan/nmorgan/chemprop/nathan/"
    )
    t0 = time.time()
    trainer.fit(mpnn, train_loader)#, val_loader)
    t1 = time.time()
    print(t1-t0)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gridsan/nmorgan/mambaforge/envs/chemprop_gpu/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
/home/gridsan/nmorgan/mambaforge/envs/chemprop_gpu/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:615: UserWarning: Checkpoint directory /home/gridsan/nmorgan/chemprop/nathan/lightning_logs/version_23666795/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-e7d358e7-783a-3a6f-9e59-ddd2825dcbd8]

  | Name      | Type             | Params
-----------------------------------------------
0 | mpn_block | BondMessageBlock | 264 K 
1 | ffn       | Sequential

Epoch 199: 100%|██████████| 1/1 [00:01<00:00,  1.42s/it, v_num=2.37e+7, train/loss=0.00632]

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it, v_num=2.37e+7, train/loss=0.00632]
303.92321610450745


In [19]:
if not torch.cuda.is_available():
    trainer = pl.Trainer(
        enable_progress_bar=True,
        devices=1,
        max_epochs=200,
        callbacks=[MyProgressBar()],
        #accelerator="gpu",
        default_root_dir="/home/gridsan/nmorgan/chemprop/nathan/"
    )
    t0 = time.time()
    trainer.fit(mpnn, train_loader)#, val_loader)
    t1 = time.time()
    print(t1-t0)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gridsan/nmorgan/mambaforge/envs/chemprop_gpu/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | mpn_block | BondMessageBlock | 264 K 
1 | ffn       | Sequential       | 90.6 K
-----------------------------------------------
354 K     Trainable params
301       Non-trainable params
355 K     Total params
1.421     Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.
/home/gridsan/nmorgan/mambaforge/envs/chemprop_gpu/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (1) is smaller

Epoch 199: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it, v_num=2.37e+7, train/loss=0.0137] 

`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 1/1 [00:01<00:00,  1.95s/it, v_num=2.37e+7, train/loss=0.0137]
355.8001642227173


In [16]:
with open("/home/gridsan/nmorgan/MW_smi.csv") as fid:
    reader = csv.reader(fid)
    next(reader)
    smis = [smi[0] for smi in reader]

pred_smis = [data.MoleculeDatapoint(smi) for smi in smis]
pred_dset = data.MoleculeDataset(pred_smis, featurizer)
pred_loader = data.MolGraphDataLoader(pred_dset, num_workers=4, shuffle=False)

In [18]:
trainer.predict(mpnn, pred_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-e7d358e7-783a-3a6f-9e59-ddd2825dcbd8]
SLURM auto-requeueing enabled. Setting signal handlers.


Predicting: 0it [00:00, ?it/s]

[(tensor([[131.2663]]), None),
 (tensor([[185.0551]]), None),
 (tensor([[424.9604]]), None),
 (tensor([[115.8282]]), None),
 (tensor([[64.1624]]), None),
 (tensor([[131.5914]]), None),
 (tensor([[64.1624]]), None),
 (tensor([[142.2961]]), None),
 (tensor([[158.8229]]), None),
 (tensor([[55.9662]]), None),
 (tensor([[384.8159]]), None),
 (tensor([[366.7430]]), None),
 (tensor([[125.0626]]), None),
 (tensor([[144.1308]]), None),
 (tensor([[112.6024]]), None),
 (tensor([[115.1009]]), None),
 (tensor([[182.9586]]), None),
 (tensor([[119.3817]]), None),
 (tensor([[32.4084]]), None),
 (tensor([[86.1871]]), None),
 (tensor([[169.4939]]), None),
 (tensor([[61.5952]]), None),
 (tensor([[112.1386]]), None),
 (tensor([[128.4797]]), None),
 (tensor([[273.5119]]), None),
 (tensor([[119.1244]]), None),
 (tensor([[171.8700]]), None),
 (tensor([[83.3038]]), None),
 (tensor([[236.8674]]), None),
 (tensor([[56.3373]]), None),
 (tensor([[198.3642]]), None),
 (tensor([[116.3813]]), None),
 (tensor([[156.2

In [22]:
checkpoint = torch.load('/home/gridsan/nmorgan/chemprop/nathan/lightning_logs/version_23652061/checkpoints/epoch=199-step=200.ckpt')
checkpoint

{'epoch': 199,
 'global_step': 200,
 'pytorch-lightning_version': '2.0.6',
 'state_dict': OrderedDict([('task_weights', tensor([[1.]])),
              ('mpn_block.cached_zero_vector',
               tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
                       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [ ]:
molenc_chk = modules.molecule_block()

In [ ]:
mpnn_chk = models.RegressionMPNN(
mpnn_chk.load_state_dict(

In [20]:
mpnn_chk = models.RegressionMPNN.load_from_checkpoint('/home/gridsan/nmorgan/chemprop/nathan/lightning_logs/version_23656678/checkpoints/epoch=199-step=200.ckpt')

/home/gridsan/nmorgan/mambaforge/envs/chemprop_gpu/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'mpn_block' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['mpn_block'])`.
  rank_zero_warn(


In [21]:
trainer.predict(mpnn_chk, pred_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-af0a01fe-fd4e-0ebe-1d1e-22a5448c6189]
SLURM auto-requeueing enabled. Setting signal handlers.


Predicting: 0it [00:00, ?it/s]

[(tensor([[133.2245]]), None),
 (tensor([[183.7518]]), None),
 (tensor([[426.3944]]), None),
 (tensor([[114.8588]]), None),
 (tensor([[64.1112]]), None),
 (tensor([[127.2082]]), None),
 (tensor([[64.1112]]), None),
 (tensor([[143.8932]]), None),
 (tensor([[161.3624]]), None),
 (tensor([[58.3730]]), None),
 (tensor([[380.3023]]), None),
 (tensor([[364.4411]]), None),
 (tensor([[124.3237]]), None),
 (tensor([[144.1082]]), None),
 (tensor([[110.7603]]), None),
 (tensor([[116.1766]]), None),
 (tensor([[181.1629]]), None),
 (tensor([[120.3119]]), None),
 (tensor([[30.7454]]), None),
 (tensor([[68.0399]]), None),
 (tensor([[170.7319]]), None),
 (tensor([[62.1280]]), None),
 (tensor([[116.1440]]), None),
 (tensor([[127.9131]]), None),
 (tensor([[283.0600]]), None),
 (tensor([[113.1282]]), None),
 (tensor([[173.4434]]), None),
 (tensor([[68.5331]]), None),
 (tensor([[253.2161]]), None),
 (tensor([[59.3567]]), None),
 (tensor([[199.6883]]), None),
 (tensor([[116.4028]]), None),
 (tensor([[158.4